In [ ]:
# Imports

import numpy as np
import time
import torch
import torch.utils.data as data_utils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
from google.colab import drive
torch.manual_seed(1000)
drive.mount('/content/gdrive')

from PIL import Image
import os

Mounted at /content/gdrive


In [ ]:
# # Do not need to run again I think

# # set the folder path
# folder_path = '/Users/richardtopaloglu/Desktop/aps360/Pneumonia-CNN-Classifier/'
# # set the size to resize the images
# size = (224, 224)

# # loop through each file in the folder
# for filename in os.listdir(folder_path):
#     if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):

#         # open the image
#         img = Image.open(os.path.join(folder_path, filename))

#         # resize the image
#         img = img.resize(size)

#         # save the image with a new filename
#         new_filename = os.path.splitext(filename)[0] + '_resized.jpg'
#         img.save(os.path.join(folder_path, new_filename))

# print('All images resized to 224 by 224')


# transform = transforms.Compose( [transforms.ToTensor()])
# test_data = torchvision.datasets.ImageFolder('/content/gdrive/My Drive/Colab Notebooks/Data/test', transform = transform)
# validate_data = torchvision.datasets.ImageFolder('/content/gdrive/My Drive/Colab Notebooks/Data/train', transform = transform)
# #train_data = torchvision.datasets.ImageFolder('/content/gdrive/My Drive/Colab Notebooks/lab3_train/',transform = transform)
# #train_loader = torch.utils.data.DataLoader(train_data, batch_size=1, num_workers=1)

Mounted at /content/gdrive


FileNotFoundError: ignored

In [ ]:
# 578 covid images
# 1585 normal images
# 4276 pneumonia images
drive.mount('/content/gdrive')

# Create data aug transformer & oversample specific classes
# to adress distribution issue

import torchvision.datasets as datasets
from torch.utils.data import DataLoader, WeightedRandomSampler

# define the parent directory where the train and test subdirectories are located
# CHANGE THIS TO MATCH YOUR PARENT DIRECTORY
parent_dir = '/content/gdrive/MyDrive/aps360/Data_new'

# define the data transformations to be applied to the images
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(size=256),
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transforms = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size=224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])



def gen_weighted_sampler(type_of, num_covid, num_normal, num_pneumonia):
  """
    type_of == 'train', 'test'

    create a sampler to try to even the distribution of the dataset wrt
    different classes.
  """

  total = num_covid + num_normal + num_pneumonia
  class_weights = [total / num_covid, total / num_normal, total / num_pneumonia]
  sample_weights = []
  for root, _, files in os.walk(os.path.join(parent_dir, type_of)):
    for filename in files:
      if "COVID" in root:
        sample_weights.append(class_weights[0])
      elif "NORMAL" in root:
        sample_weights.append(class_weights[1])
      else:
        sample_weights.append(class_weights[2])
  sample_weights = torch.from_numpy(np.array(sample_weights))
  sampler = WeightedRandomSampler(sample_weights, len(sample_weights))
  return sampler




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:

# Apply data augmentation techniques on dataset

######## train #############
train_data = datasets.ImageFolder(os.path.join(parent_dir, 'train'), transform=train_transforms)

# set color_mode to 'RGB'
train_data.color_mode = 'RGB'

# set class_mode to 'categorical'
train_data.class_mode = 'categorical'

# image counts in training folder
num_covid = 461
num_normal = 3240
num_pneumonia = 1267

train_sampler = gen_weighted_sampler('train', num_covid, num_normal, num_pneumonia)
train_loader = DataLoader(train_data, batch_size=32, sampler=train_sampler, num_workers=4)

###########################

####### test ###############
test_data = datasets.ImageFolder(os.path.join(parent_dir, 'test'), transform=test_transforms)

# set color_mode to 'RGB'
test_data.color_mode = 'RGB'

# set class_mode to 'categorical'
test_data.class_mode = 'categorical'

# image counts in test folder
num_test_covid = 117
num_test_normal = 318
num_test_pneumonia = 856

test_sampler = gen_weighted_sampler('test', num_test_covid, num_test_normal, num_test_pneumonia)
test_loader = DataLoader(test_data, batch_size=32, sampler=test_sampler, num_workers=4)

############################







FileNotFoundError: ignored

['COVID19', 'NORMAL', 'PNEUMONIA']